# Experiment 1: Hello World Rerun

## Research Question
Is the implementation of workflow reproduciblity in DALiuGE capable of successfully finding two rerun graphs

## Variables
|Variable Type|Explanation                                                     |
|:------------:|:---------------------------------------------------------------:|
|Independent  |Parameters of each logical graph (sumamrised to a file location)|
|Dependent    |Merklehash value of leaf nodes in graph                         |
|Confounding  |Runtime failures                                                |

In [1]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import json
import optparse
import subprocess
from IPython.display import display, display_pretty, display_json
from dlg.common import tool
from dlg.common.reproducibility.reproducibility import ReproducibilityFlags
from dlg.translator.tool_commands import dlg_fill, dlg_unroll, dlg_partition, dlg_map, dlg_submit

In [2]:
def run_full_workflow(workflow: str, rmode: ReproducibilityFlags):
    lgt = workflow + ".graph"
    lg = workflow + "LG.graph"
    pgs = workflow + "PGS.graph"
    pgt = workflow + "PGT.graph"
    pg = workflow + "PG.graph"
    rg = workflow + ".out"
    daemon = tool.start_process('daemon', ['-mm'], stdout=subprocess.PIPE)

    rmodes = str(rmode.value)

    parser = optparse.OptionParser()
    dlg_fill(parser, ['-L', lgt, '-R', rmodes, '-o', lg, '-f', 'newline'])
    parser = optparse.OptionParser()
    dlg_unroll(parser, ['-L', lg, '-o', pgs, '-f', 'newline'])
    parser = optparse.OptionParser()
    dlg_partition(parser, ['-P', pgs, '-o', pgt, '-f', 'newline'])
    parser = optparse.OptionParser()
    dlg_map(parser, ['-P', pgt, '-N', '127.0.0.1,127.0.0.1', '-o', pg, '-f', 'newline'])
    parser = optparse.OptionParser()
    dlg_submit(parser, ['-P', pg, '-p', '8000', '-R', '-o', rg, '-f', 'newline'])
    daemon.kill()

def test_identical(w1: str, w2: str):
    f1 = open(w1 + '.out')
    f2 = open(w2 + '.out')
    r1 = json.load(f1)
    r2 = json.load(f2)
    f1.close()
    f2.close()

    return r1['reprodata']['signature'] == r2['reprodata']['signature']

def summarise_run(record: dict):
    return {'result_hash': record['reprodata']['leaves'], 'meta_data': record['reprodata']['meta_data'],
            'meta_merkleroot': record['reprodata']['merkleroot']}

In [3]:
graph1_widget = widgets.Text(value='HelloSBash')
graph2_widget = widgets.Text(value='HelloWorldBash')
display(graph1_widget)
display(graph2_widget)

Text(value='HelloSBash')

Text(value='HelloWorldBash')

In [4]:
run_workflow = widgets.Checkbox(value=False, description='Run graph files?', indent=False)
display(run_workflow)

Checkbox(value=False, description='Run graph files?', indent=False)

In [5]:
rmode_widget = widgets.Dropdown(
    options={'Rerun': ReproducibilityFlags.RERUN,\
             'Repeat': ReproducibilityFlags.REPEAT, \
             'Nothing': ReproducibilityFlags.NOTHING
            }
)
display(rmode_widget)

Dropdown(options={'Rerun': <ReproducibilityFlags.RERUN: 1>, 'Repeat': <ReproducibilityFlags.REPEAT: 2>, 'Nothi…

In [6]:
result_widget = widgets.Text(value='')
display(result_widget)

Text(value='')

In [7]:
run_button = widgets.Button(description="RUN!")
display(run_button)

def run(g1, g2, rmode, run=False):
    if run:
        result_widget.value = "Running workflow %s.graph" % graph1_widget.value
        run_full_workflow(g1, rmode)
        result_widget.value = "Running workflow %s.graph" % graph2_widget.value
        run_full_workflow(g2, rmode)
    result = test_identical(g1, g2)
    if result:
        result_widget.value = 'SUCCESS'
    else:
        result_widget.value = 'FAILURE'

def button_click(self):
    run(graph1_widget.value, graph2_widget.value, rmode_widget.value, run_workflow.value)

run_button.on_click(button_click)

Button(description='RUN!', style=ButtonStyle())